In [1]:
import os
import pytz
from datetime import datetime, timedelta

import yfinance as yf
from hemlock import User, Page, create_test_app
from hemlock.questions import Input, Label
from hemlock_ax import Assigner, init_test_app
from flask_login import current_user

app = create_test_app()
init_test_app(app)

In [2]:
DATE_FORMAT = "%Y-%m-%d"

def get_crude_oil_price():
    df = yf.Ticker("CL=F").history("5d").reset_index().iloc[-2]
    return df.Date.strftime(DATE_FORMAT), df.Close

def get_s_and_p():
    df = yf.Ticker("^GSPC").history("5d").reset_index().iloc[-2]
    return df.Date.strftime(DATE_FORMAT), df.Close

forecast_questions = [
    (
        "crude",
        """
        What do you think the price of Crude Oil will be at the end of the day today?
        
        You can find a graph of recent Crude Oil prices
        <a href="https://finance.yahoo.com/chart/CL%3DF" target="_blank">here</a>.
        """,
        get_crude_oil_price
    ),
    (
        "s_and_p",
        """
        What do you think the S&P 500 index will close at today?

        You can find a graph of recent values of the S&P 500 index
        <a href="https://finance.yahoo.com/chart/%5EGSPC" target="_blank">here</a>.
        """,
        get_s_and_p
    )
]

get_crude_oil_price()

('2022-03-11', 109.33000183105469)

In [3]:
assigner = Assigner({"treatment": (0, 1, 2)})

In [4]:
def seed():
    """Creates the main survey branch.

    Returns:
        List[Page]: List of pages shown to the user.
    """
    
    if os.getenv("FLASK_ENV") == "production":
        start = current_user.start_time
        current_user.meta_data["start_time_et"] = (
            datetime(start.year, start.month, start.day, tzinfo=pytz.utc)
            .astimezone(pytz.timezone("US/Eastern"))
            .strftime(DATE_FORMAT)
        )
    else:
        current_user.meta_data["start_time_et"] = forecast_questions[0][2]()[0]

    assignment = assigner.assign_user()
    current_user.meta_data["question"] = [q[0] for q in forecast_questions]
    current_user.meta_data["outcome"] = len(forecast_questions) * [None]
    current_user.meta_data["target"] = len(forecast_questions) * [None]
    forecast_pages = [
        Page(
            Label(
                f"You were assigned to {assignment}."
            ),
            Input(
                q[1],
                input_tag={"type": "number", "required": True},
                variable="forecast"
            )
        )
        for q in forecast_questions
    ]
    return forecast_pages + [
        Page(
            Label("Thanks for participating!"),
        )
    ]

user = User.make_test_user(seed)
user.test(verbosity=1)

Traceback (most recent call last):
  File "C:\Users\DBSpe\anaconda3\envs\sota\lib\site-packages\hemlock_ax\assign.py", line 254, in fit_model
    return model(self.get_data(self), **self.model_kwargs)
  File "C:\Users\DBSpe\anaconda3\envs\sota\lib\site-packages\hemlock_ax\models.py", line 40, in linear_regression
    df = df.dropna(subset=exog_names + [endog_name])
  File "C:\Users\DBSpe\anaconda3\envs\sota\lib\site-packages\pandas\util\_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\DBSpe\anaconda3\envs\sota\lib\site-packages\pandas\core\frame.py", line 5996, in dropna
    raise KeyError(np.array(subset)[check].tolist())
KeyError: ['treatment', 'target']
C:\Users\DBSpe\anaconda3\envs\sota\lib\site-packages\hemlock_ax\assign.py:257: RuntimeWarning: MODEL FAILED TO FIT
This warning is normal early in the study when there are not enough data to fit the model. If you continue to see this warning, there is likely a more serious error.

  warnings.war

In [5]:
user.get_data()

,id,completed,failed,errored,in_progress,start_time,end_time,total_seconds,start_time_et,treatment,question,outcome,target,forecast
0,1,True,False,False,False,2022-03-14 23:11:45.789779,2022-03-14 23:11:46.856605,1.066826,2022-03-11,1,crude,None,None,-9001.0
1,1,True,False,False,False,2022-03-14 23:11:45.789779,2022-03-14 23:11:46.856605,1.066826,2022-03-11,1,s_and_p,None,None,675.0


In [6]:
User.get_all_data()

,id,completed,failed,errored,in_progress,start_time,end_time,total_seconds,start_time_et,treatment,question,outcome,target,forecast
0,1,True,False,False,False,2022-03-14 23:11:45.789779,2022-03-14 23:11:46.856605,1.066826,2022-03-11,1,crude,None,None,-9001.0
1,1,True,False,False,False,2022-03-14 23:11:45.789779,2022-03-14 23:11:46.856605,1.066826,2022-03-11,1,s_and_p,None,None,675.0


In [7]:
user = User.make_test_user(seed)
user.test_get()

Traceback (most recent call last):
  File "C:\Users\DBSpe\anaconda3\envs\sota\lib\site-packages\hemlock_ax\assign.py", line 254, in fit_model
    return model(self.get_data(self), **self.model_kwargs)
  File "C:\Users\DBSpe\anaconda3\envs\sota\lib\site-packages\hemlock_ax\models.py", line 42, in linear_regression
    model = sm.OLS(df[endog_name], pd.get_dummies(assignments))
  File "C:\Users\DBSpe\anaconda3\envs\sota\lib\site-packages\statsmodels\regression\linear_model.py", line 890, in __init__
    super(OLS, self).__init__(endog, exog, missing=missing,
  File "C:\Users\DBSpe\anaconda3\envs\sota\lib\site-packages\statsmodels\regression\linear_model.py", line 717, in __init__
    super(WLS, self).__init__(endog, exog, missing=missing,
  File "C:\Users\DBSpe\anaconda3\envs\sota\lib\site-packages\statsmodels\regression\linear_model.py", line 191, in __init__
    super(RegressionModel, self).__init__(endog, exog, **kwargs)
  File "C:\Users\DBSpe\anaconda3\envs\sota\lib\site-packages\sta

<Tree id: 2>
    <Page 0>
        <Label You were assigned to {'treatment': 0}. - default: None>
        <Input What do you think the price of [...] - default: None>
    <Page 1>
        <Label You were assigned to {'treatment': 0}. - default: None>
        <Input What do you think the S&P 500 [...] - default: None>
    <Page 2 terminal>
        <Label Thanks for participating! - default: None>

In [8]:
user.get_data()

,id,completed,failed,errored,in_progress,start_time,end_time,total_seconds
0,2,False,False,False,True,2022-03-14 23:12:12.299339,2022-03-14 23:12:12.299339,0.0


In [9]:
User.get_all_data()

,id,completed,failed,errored,in_progress,start_time,end_time,total_seconds,start_time_et,treatment,question,outcome,target,forecast
0,1,True,False,False,False,2022-03-14 23:11:45.789779,2022-03-14 23:11:46.856605,1.066826,2022-03-11,1.0,crude,None,None,-9001.0
1,1,True,False,False,False,2022-03-14 23:11:45.789779,2022-03-14 23:11:46.856605,1.066826,2022-03-11,1.0,s_and_p,None,None,675.0
2,2,False,False,False,True,2022-03-14 23:12:12.299339,2022-03-14 23:12:12.299339,0.000000,None,NaN,None,None,None,NaN


In [10]:
user._cached_data

{'id': [2],
 'completed': [False],
 'failed': [False],
 'errored': [False],
 'in_progress': [True],
 'start_time': ['2022-03-14 23:12:12.299339'],
 'end_time': ['2022-03-14 23:12:12.299339'],
 'total_seconds': [0.0]}

In [10]:
user.test_request([None, 100])

INFO:root:<Page 0>
    <Label You were assigned to {'treatment': 0}. - default: None>
        test response: None
    <Input What do you think the price of [...] - default: None>
        test response: 100
    test direction: 'forward'


<Tree id: 2>
    <Page 0>
        <Label You were assigned to {'treatment': 0}. - response: None>
        <Input What do you think the price of [...] - response: 100.0>
    <Page 1>
        <Label You were assigned to {'treatment': 0}. - default: None>
        <Input What do you think the S&P 500 [...] - default: None>
    <Page 2 terminal>
        <Label Thanks for participating! - default: None>

In [11]:
user.get_data()

,id,completed,failed,errored,in_progress,start_time,end_time,total_seconds
0,2,False,False,False,True,2022-03-14 22:59:09.847299,2022-03-14 22:59:09.847299,0.0


In [12]:
User.get_all_data()

padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data is None
padding data 

,id,completed,failed,errored,in_progress,start_time,end_time,total_seconds,start_time_et,treatment,question,outcome,target,forecast
0,1,True,False,False,False,2022-03-14 22:58:40.568690,2022-03-14 22:58:41.574376,1.005686,2022-03-11,2.0,crude,None,None,NaN
1,1,True,False,False,False,2022-03-14 22:58:40.568690,2022-03-14 22:58:41.574376,1.005686,2022-03-11,2.0,s_and_p,None,None,9152.0
2,2,False,False,False,True,2022-03-14 22:59:09.847299,2022-03-14 22:59:09.847299,0.000000,None,NaN,None,None,None,NaN


In [ ]:
# You can easily and quickly test new code in a notebook
# before adding it to the survey (the __ini__.py file)
import os

import matplotlib.pyplot as plt
import seaborn as sns
from hemlock import create_test_app
from hemlock_ax import init_test_app, run_test

import src

# remove GITPOD_HOST from environment
# this is used for redirects when running an app in gitpod
# but here we're only running a test app
os.environ.pop("GITPOD_HOST", None)

sns.set()
app = create_test_app()
init_test_app(app)
df = run_test(100)[0]
df.head()

In [ ]:
sns.lineplot(data=df, x="n_assigned_users", y="pr_best", hue="assignment")
plt.legend(bbox_to_anchor=(1,1), loc="upper left")
plt.show()

In [ ]:
sns.lineplot(data=df, x="n_assigned_users", y="weight", hue="assignment")
plt.legend(bbox_to_anchor=(1,1), loc="upper left")
plt.show()

In [ ]:
sns.lineplot(data=df, x="n_assigned_users", y="cum_assigned", hue="assignment")
plt.legend(bbox_to_anchor=(1,1), loc="upper left")
plt.show()